In [2]:
import pandas as pd

In [8]:
data = pd.read_csv(r'D:\DATA KANTOR\SQL\13122021_PPMPKM.csv',sep=',', dtype={'nosk':'str','NPWP':'str','KPP':'str','CABANG':'str'})

In [9]:
data = data[data['BULANBAYAR']==12]
data['NPWP_FULL'] = data.NPWP+data.KPP+data.CABANG
ppm = data[data['FLAG_PPM_PKM']=='PPM']
pkm = data[data['FLAG_PPM_PKM']=='PKM']

In [10]:
pkm['nosk'] = pkm['nosk'].replace('-','')
pkm['nosk'] = pkm['nosk'].replace('tidak diketahui','')

<ipython-input-10-9a960c5d97bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pkm['nosk'] = pkm['nosk'].replace('-','')
<ipython-input-10-9a960c5d97bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pkm['nosk'] = pkm['nosk'].replace('tidak diketahui','')


In [11]:
pengawasan = pkm[pkm['nosk'].isnull()]
pengawasan['Jenis'] ='Pengawasan'

<ipython-input-11-f8d01815a471>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pengawasan['Jenis'] ='Pengawasan'


In [12]:
p2 = pkm[pkm['nosk'].notnull()]

In [13]:
p2['Jenis'] = p2['nosk'].apply(lambda x:'Pemeriksaan' if x[-2:]=='21' else 'Penagihan')

<ipython-input-13-5b19ada81123>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p2['Jenis'] = p2['nosk'].apply(lambda x:'Pemeriksaan' if x[-2:]=='21' else 'Penagihan')


In [14]:
p2.Jenis.value_counts()

Penagihan      238
Pemeriksaan    167
Name: Jenis, dtype: int64

In [15]:
ppm['Jenis'] = 'Rutin'

<ipython-input-15-9d35d00a8f6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppm['Jenis'] = 'Rutin'


In [16]:
gabung = p2.append(pengawasan)
gabung = gabung.append(ppm)

In [17]:
gabung.Jenis.value_counts()

Rutin          11000
Penagihan        238
Pengawasan       230
Pemeriksaan      167
Name: Jenis, dtype: int64

In [ ]:
gabung.to_excel('D:\DATA KANTOR\SQL\Desember.xlsx',index=False, engine='openpyxl')

In [ ]:
gabung['Jenis'].value_counts()

In [186]:
prk = gabung[gabung['Jenis']=='Pemeriksaan']
prk = prk.groupby(['NPWP_FULL','NAMA']).sum().reset_index()

In [189]:
prk.to_clipboard(sep='|')

In [182]:
slice = gabung.filter(['NPWP_FULL','NOMINAL','Jenis'])
for jenis in slice['Jenis'].unique():
    temp = slice[slice['Jenis']==jenis]
    temp = temp.groupby('NPWP_FULL').sum()
    temp.to_excel('{}.xlsx'.format(jenis),index='False')
    


In [ ]:
slice = gabung.filter(['NPWP_FULL','NOMINAL','Jenis'])
rutin = slice[slice['Jenis']=='Rutin']
rutin = rutin.groupby('NPWP_FULL').sum().reset_index()
rutin.to_excel('Rutin.xlsx',index=False,engine='openpyxl')

pemeriksaan = slice[slice['Jenis']=='Pemeriksaan']
pemeriksaan = pemeriksaan.groupby('NPWP_FULL').sum().reset_index()
pemeriksaan.to_excel('Pemeriksaan.xlsx',index=False,engine='openpyxl')

penagihan = slice[slice['Jenis']=='Penagihan']
penagihan = penagihan.groupby('NPWP_FULL').sum().reset_index()
penagihan.to_excel('Penagihan.xlsx',index=False,engine='openpyxl')

pengawasan = slice[slice['Jenis']=='Pengawasan']
pengawasan = pengawasan.groupby('NPWP_FULL').sum().reset_index()
pengawasan.to_excel('Pengawasan.xlsx',index=False,engine='openpyxl')

In [191]:
import mysql.connector as mysql

In [193]:
db_conn = mysql.connect(host = '10.4.19.15', user = 'indra', password= 'indra',
database = 'mpninfo',port = '3306')
with open(r'F:\STREAMLIT\DASHTEAM\sql\mpnspmpbkspmkp.sql','r') as query:
    penerimaan = query.read()
data = pd.read_sql(penerimaan,db_conn)

In [216]:
data = data[data['bulanbayar']==10]

In [217]:
spmkp = data[data['ket']=='SPMKP']

In [218]:
spmkp.columns

Index(['NPWP_FULL', 'nama', 'kdmap', 'kdbayar', 'masa', 'tahun',
       'tanggalbayar', 'bulanbayar', 'tahunbayar', 'datebayar', 'nominal',
       'ntpn', 'bank', 'nosk', 'nospm', 'tipe', 'source', 'extra', 'billing',
       'nop', 'pembuat', 'ket'],
      dtype='object')

In [221]:
spmkp = spmkp.filter(['NPWP_FULL','nominal','nama'])
spmkp = spmkp.groupby('NPWP_FULL').sum().reset_index()

In [225]:
spmkp.to_clipboard(sep='|')